# PegasosQSVC - DGA Botnet detection

Madjid Tehrani
Cybersec-DMS (DMS GmbH)
#https://www.cybersec-dms.com/
Copyright © 2023, DMS GmbH. All rights reserved. Permission is hereby granted, free of charge, to any person obtaining a copy of this
software and associated documentation files (the "Software"), for academic research purposes, subject to the following conditions:
the above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.   


In [ ]:
!pip install pylatexenc
!pip install qiskit --upgrade;
!pip install qiskit-aer;
!pip install qiskit-terra;
!pip install -U azure-quantum
!pip install -U azure-quantum[qiskit]
!pip install -U qiskit_machine_learning

In [ ]:
#Function to load a file from a publicly accessible URL
import urllib.request

def download_file(url, save_path):
    urllib.request.urlretrieve(url, save_path)
    print(f'File successfully downloaded to {save_path}.')

In [ ]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from azure.quantum.qiskit import AzureQuantumProvider
from qiskit import QuantumCircuit, execute, Aer
from qiskit import *
from qiskit.utils import QuantumInstance
from azure.quantum import Workspace

provider = AzureQuantumProvider (
    resource_id = "/subscriptions/YOUR-ID/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/YourWorkspace",
    location = "eastus"
)

print("This workspace's targets:")
for backend in provider.backends():
    print("- " + backend.name())

workspace = Workspace (
    subscription_id = "YOUR-ID",
    resource_group = "AzureQuantum",
    name = "YourWorkspace",
    location = "eastus"

)

Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.


This workspace's targets:


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code R65SR24GU to authenticate.
- ionq.simulator
- ionq.simulator
- ionq.simulator
- ionq.qpu
- ionq.qpu
- ionq.qpu
- ionq.qpu.aria-1
- ionq.qpu.aria-1
- ionq.qpu.aria-1
- quantinuum.hqs-lt-s1-apival
- quantinuum.hqs-lt-s1-apival
- quantinuum.sim.h1-1sc
- quantinuum.sim.h1-1sc
- quantinuum.hqs-lt-s2-apival
- quantinuum.hqs-lt-s2-apival
- quantinuum.sim.h1-2sc
- quantinuum.sim.h1-2sc
- quantinuum.hqs-lt-s1-sim
- quantinuum.hqs-lt-s1-sim
- quantinuum.sim.h1-1e
- quantinuum.sim.h1-1e
- quantinuum.hqs-lt-s2-sim
- quantinuum.hqs-lt-s2-sim
- quantinuum.sim.h1-2e
- quantinuum.sim.h1-2e
- quantinuum.hqs-lt-s1
- quantinuum.hqs-lt-s1
- quantinuum.qpu.h1-1
- quantinuum.qpu.h1-1
- quantinuum.hqs-lt-s2
- quantinuum.hqs-lt-s2
- quantinuum.qpu.h1-2
- quantinuum.qpu.h1-2
- rigetti.sim.qvm
- rigetti.qpu.aspen-m-3
- microsoft.estimator


In [ ]:
# Create IonQ simulator and QPU backends
ionq_simulator_backend = provider.get_backend("ionq.simulator")
ionq_qpu_backend = provider.get_backend("ionq.qpu")

# Create Rigetti simulator and QPU backends
rigetti_simulator_backend = provider.get_backend("rigetti.sim.qvm")
rigetti_qpu_backend = provider.get_backend("rigetti.qpu.aspen-m-3")

# Create Quantinuum simulator and QPU backends
Quantinuum_simulator_backend = provider.get_backend("quantinuum.sim.h1-2sc")
Quantinuum_qpu_backend = provider.get_backend("quantinuum.qpu.h1-2")

# Create QASM simulator
backend = Aer.get_backend('qasm_simulator')

In [ ]:
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap, ZFeatureMap
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.circuit.library import RawFeatureVector
from qiskit.algorithms.optimizers import SLSQP, SPSA
from qiskit.circuit.library import TwoLocal
from qiskit import Aer, transpile

import numpy as np
import matplotlib.pyplot as plt
import datetime
import concurrent.futures
import time
import os
import csv
import pandas as pd
from IPython.display import clear_output

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Binarizer
from sklearn.decomposition import PCA


n_features = 7
training_size = 700
test_size = 300
feature_dim = 7
num_samples = 1000
datafilename="BotnetDgaDataset_1000.csv"
resultname="result_BotnetDgaDataset_1000.txt"
cwd=os.getcwd()
mycsv=cwd+"/"+datafilename

download_file('https://aq5efd7d2644dd406cb3ec2d.blob.core.windows.net/dga/BotnetDgaDataset_1000.csv', mycsv)

def load_data(filepath):

    with open(filepath) as csv_file:
        data_file = csv.reader(csv_file)
        temp = next(data_file)
        n_samples = 1000
        n_features = 7
        target_names = np.array(temp[2:])
        data = np.empty((n_samples, n_features))
        target = np.empty((n_samples,), dtype=np.int)

        for i, ir in enumerate(data_file):
            data[i] = np.asarray(ir[:-1], dtype=np.float64)
            target[i] = np.asarray(ir[-1], dtype=np.int)

    return data, target, target_names

File successfully downloaded to /content/BotnetDgaDataset_1000.csv.


In [ ]:
#code from https://ieee-dataport.org/open-access/botnet-dga-dataset
def _convert_data_dataframe(data, target,
                            feature_names, target_names):
    data_df = pd.DataFrame(data, columns=feature_names)
    target_df = pd.DataFrame(target, columns=target_names)
    combined_df = pd.concat([data_df, target_df], axis=1)
    X = combined_df[feature_names]
    y = combined_df[target_names]
    if y.shape[1] == 1:
        y = y.iloc[:, 0]
    return combined_df, X, y


def load_botnetdga(*, as_frame=False):

    data, target, target_names = load_data('BotnetDgaDataset_1000.csv')



    feature_names = ['MinREBotnets',
                     'CharLength',
                     'TreeNewFeature',
                     'nGramReputation_Alexa']

    frame = None
    target_columns = ['target', ]
    if as_frame:
        frame, data, target = _convert_data_dataframe(data,
                                                      target,
                                                      feature_names,
                                                      target_columns)

    return data, target


def botnetdga(training_size, test_size, n,
              standardize=False, pca=False, scale=False, plot_data=False,
              binarize=False):

    class_labels = [r'benign', r'dga']

    data, target = load_botnetdga()
    sample_train, sample_test, label_train, label_test = \
        train_test_split(data, target, train_size=training_size, test_size=test_size, random_state=7)

    # print("\n load_botnetdga sample_train = \n")
    # print(sample_train)
    # print("\n load_botnetdga sample_test = \n")
    # print(sample_test)

    # Now we standardize for gaussian around 0 with unit variance
    if standardize:
        std_scale = StandardScaler().fit(sample_train)
        sample_train = std_scale.transform(sample_train)
        sample_test = std_scale.transform(sample_test)

        # print("\n standardize sample_train = \n")
        # print(sample_train)
        # print("\n standardize sample_test = \n")
        # print(sample_test)

    # Now reduce number of features to number of qubits
    if pca:
        pca = PCA(n_components=n).fit(sample_train)
        sample_train = pca.transform(sample_train)
        sample_test = pca.transform(sample_test)

        # print("\n pca sample_train = \n")
        # print(sample_train)
        # print("\n pca sample_test = \n")
        # print(sample_test)

    # Scale to the range (-1,+1)
    if scale:
        samples = np.append(sample_train, sample_test, axis=0)
        minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
        sample_train = minmax_scale.transform(sample_train)
        sample_test = minmax_scale.transform(sample_test)

        # print("\n scale sample_train = \n")
        # print(sample_train)
        # print("\n scale sample_test = \n")
        # print(sample_test)

    if binarize:
        med = np.median(np.append(sample_train, sample_test, axis=0), axis=0)

        # print("\n binarize np.append(sample_train, sample_test, axis=0) = \n")
        # print(np.append(sample_train, sample_test, axis=0)[:5])
        # print("\n binarize med = \n")
        # print(med)

        transformer = Binarizer(threshold=med)
        # print("\n binarize transformer = \n")
        # print(transformer)

        sample_train = transformer.transform(sample_train)
        sample_test = transformer.transform(sample_test)

        # print("\n binarize sample_train = \n")
        # print(sample_train)
        # print("\n binarize sample_test = \n")
        # print(sample_test)

    # Pick training size number of samples from each distro
    training_input = {key: (sample_train[label_train == k, :])[:training_size]
                      for k, key in enumerate(class_labels)}
    test_input = {key: (sample_test[label_test == k, :])[:test_size]
                  for k, key in enumerate(class_labels)}

    return sample_train, training_input, test_input, class_labels

In [ ]:
#import time
start = time.perf_counter()

# BotnetDGA data set
plot_data = False
training_size = 700
test_size = 300
feature_dim = 7
standardize = False
pca = False
scale = False
binarize = False



# random_seed = 10598
# shots = 1024
# seed = 1376
# aqua_globals.random_seed = seed

sample_train, training_input, test_input, class_labels = botnetdga(training_size=training_size,
                                                                    test_size=test_size,
                                                                    n=feature_dim,
                                                                    standardize=standardize,
                                                                    pca=pca,
                                                                    scale=scale,
                                                                    plot_data=plot_data,
                                                                    binarize=binarize)

# print("\n sample_train = \n")
# print(sample_train)
# print("\n training_input = \n")
# print(training_input)
# print("\n test_input = \n")
# print(test_input)
# print("\n class_labels = \n")
# print(class_labels)

optimizer1 = SPSA()
#optimizer2 = ADAM()
#optimizer3 = AQGD()
#optimizer4 = CG()
#optimizer5 = COBYLA()
#optimizer6 = L_BFGS_B()
#optimizer7 = GSLS()
#optimizer8 = NELDER_MEAD()
#optimizer9 = NFT()
#optimizer10 = P_BFGS()
#optimizer11 = POWELL()
#optimizer12 = SLSQP()
#optimizer13 = TNC()

nFeature = 7

#feature_map1 = RawFeatureVector(nFeature)
#feature_map2 = PauliFeatureMap(nFeature)
feature_map3 = ZFeatureMap(nFeature)
#feature_map4 = ZZFeatureMap(nFeature)

#var_form11 = TwoLocal(feature_map1.num_qubits, ['ry', 'rz'], 'cz')
#var_form12 = RealAmplitudes(feature_map1.num_qubits)
#var_form13 = EfficientSU2(feature_map1.num_qubits)
#var_form14 = ExcitationPreserving(feature_map1.num_qubits)

#var_form21 = TwoLocal(feature_map2.num_qubits, ['ry', 'rz'], 'cz')
#var_form22 = RealAmplitudes(feature_map2.num_qubits)
#var_form23 = EfficientSU2(feature_map2.num_qubits)
#var_form24 = ExcitationPreserving(feature_map2.num_qubits)

#var_form31 = TwoLocal(feature_map3.num_qubits, ['ry', 'rz'], 'cz')
var_form32 = RealAmplitudes(feature_map3.num_qubits)
#var_form33 = EfficientSU2(feature_map3.num_qubits)
#var_form34 = ExcitationPreserving(feature_map3.num_qubits)

#var_form41 = TwoLocal(feature_map4.num_qubits, ['ry', 'rz'], 'cz')
#var_form42 = RealAmplitudes(feature_map4.num_qubits)
#var_form43 = EfficientSU2(feature_map4.num_qubits)
#var_form44 = ExcitationPreserving(feature_map4.num_qubits)

In [ ]:
# This code is part of Qiskit. But to handle exception we changed it. Yuo dont need it if you use AER. This is an acedemic test example and we dont intend to change the code out of its license, however to run over other simulators and real devices its a neccessary change.

"""Pegasos Quantum Support Vector Classifier."""
import time
import signal


from __future__ import annotations

import logging
from datetime import datetime
from typing import Dict

import numpy as np
from qiskit.utils import algorithm_globals
from sklearn.base import ClassifierMixin

from qiskit_machine_learning.algorithms  import SerializableModelMixin
from qiskit_machine_learning import QiskitMachineLearningError
from qiskit_machine_learning.kernels import FidelityQuantumKernel, QuantumKernel, BaseKernel


logger = logging.getLogger(__name__)


class PegasosQSVC(ClassifierMixin, SerializableModelMixin):
    r"""
    Implements Pegasos Quantum Support Vector Classifier algorithm. The algorithm has been
    developed in [1] and includes methods ``fit``, ``predict`` and ``decision_function`` following
    the signatures
    of `sklearn.svm.SVC <https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html>`_.
    This implementation is adapted to work with quantum kernels.

    **Example**

    .. code-block:: python

        quantum_kernel = FidelityQuantumKernel()

        pegasos_qsvc = PegasosQSVC(quantum_kernel=quantum_kernel)
        pegasos_qsvc.fit(sample_train, label_train)
        pegasos_qsvc.predict(sample_test)

    **References**
        [1]: Shalev-Shwartz et al., Pegasos: Primal Estimated sub-GrAdient SOlver for SVM.
            `Pegasos for SVM <https://home.ttic.edu/~nati/Publications/PegasosMPB.pdf>`_

    """

    FITTED = 0
    UNFITTED = 1
    # pylint: disable=invalid-name
    def __init__(
        self,
        quantum_kernel: BaseKernel | None = None,
        C: float = 1.0,
        num_steps: int = 1000,
        precomputed: bool = False,
        seed: int | None = None,
    ) -> None:
        """
        Args:
            quantum_kernel: a quantum kernel to be used for classification. Has to be ``None`` when
                a precomputed kernel is used.
            C: Positive regularization parameter. The strength of the regularization is inversely
                proportional to C. Smaller ``C`` induce smaller weights which generally helps
                preventing overfitting. However, due to the nature of this algorithm, some of the
                computation steps become trivial for larger ``C``. Thus, larger ``C`` improve
                the performance of the algorithm drastically. If the data is linearly separable
                in feature space, ``C`` should be chosen to be large. If the separation is not
                perfect, ``C`` should be chosen smaller to prevent overfitting.

            num_steps: number of steps in the Pegasos algorithm. There is no early stopping
                criterion. The algorithm iterates over all steps.
            precomputed: a boolean flag indicating whether a precomputed kernel is used. Set it to
                ``True`` in case of precomputed kernel.
            seed: a seed for the random number generator

        Raises:
            ValueError:
                - if ``quantum_kernel`` is passed and ``precomputed`` is set to ``True``. To use
                a precomputed kernel, ``quantum_kernel`` has to be of the ``None`` type.
            TypeError:
                - if ``quantum_kernel`` neither instance of
                  :class:`~qiskit_machine_learning.kernels.BaseKernel` nor ``None``.
        """

        if precomputed:
            if quantum_kernel is not None:
                raise ValueError("'quantum_kernel' has to be None to use a precomputed kernel")
        else:
            if quantum_kernel is None:
                quantum_kernel = FidelityQuantumKernel()

        self._quantum_kernel = quantum_kernel
        self._precomputed = precomputed
        self._num_steps = num_steps
        if seed is not None:
            algorithm_globals.random_seed = seed

        if C > 0:
            self.C = C
        else:
            raise ValueError(f"C has to be a positive number, found {C}.")

        # these are the parameters being fit and are needed for prediction
        self._alphas: Dict[int, int] | None = None
        self._x_train: np.ndarray | None = None
        self._n_samples: int | None = None
        self._y_train: np.ndarray | None = None
        self._label_map: Dict[int, int] | None = None
        self._label_pos: int | None = None
        self._label_neg: int | None = None

        # added to all kernel values to include an implicit bias to the hyperplane
        self._kernel_offset = 1

        # for compatibility with the base SVC class. Set as unfitted.
        self.fit_status_ = PegasosQSVC.UNFITTED

    # pylint: disable=invalid-name
    def fit(
        self, X: np.ndarray, y: np.ndarray, sample_weight: np.ndarray | None = None
    ) -> "PegasosQSVC":
        """Fit the model according to the given training data.

        Args:
            X: Train features. For a callable kernel (an instance of
               :class:`~qiskit_machine_learning.kernels.BaseKernel`) the shape
               should be ``(n_samples, n_features)``, for a precomputed kernel the shape should be
               ``(n_samples, n_samples)``.
            y: shape (n_samples), train labels . Must not contain more than two unique labels.
            sample_weight: this parameter is not supported, passing a value raises an error.

        Returns:
            ``self``, Fitted estimator.

        Raises:
            ValueError:
                - X and/or y have the wrong shape.
                - X and y have incompatible dimensions.
                - y includes more than two unique labels.
                - Pre-computed kernel matrix has the wrong shape and/or dimension.

            NotImplementedError:
                - when a sample_weight which is not None is passed.
        """
        # check whether the data have the right format
        if np.ndim(X) != 2:
            raise ValueError("X has to be a 2D array")
        if np.ndim(y) != 1:
            raise ValueError("y has to be a 1D array")
        if len(np.unique(y)) != 2:
            raise ValueError("Only binary classification is supported")
        if X.shape[0] != y.shape[0]:
            raise ValueError("'X' and 'y' have to contain the same number of samples")
        if self._precomputed and X.shape[0] != X.shape[1]:
            raise ValueError(
                "For a precomputed kernel, X should be in shape (n_samples, n_samples)"
            )
        if sample_weight is not None:
            raise NotImplementedError(
                "Parameter 'sample_weight' is not supported. All samples have to be weighed equally"
            )
        # reset the fit state
        self.fit_status_ = PegasosQSVC.UNFITTED

        # the algorithm works with labels in {+1, -1}
        self._label_pos = np.unique(y)[0]
        self._label_neg = np.unique(y)[1]
        self._label_map = {self._label_pos: +1, self._label_neg: -1}

        # the training data are later needed for prediction
        self._x_train = X
        self._y_train = y
        self._n_samples = X.shape[0]

        # empty dictionary to represent sparse array
        self._alphas = {}

        t_0 = datetime.now()
        # training loop
        for step in range(1, self._num_steps + 1):
            # for every step, a random index (determining a random datum) is fixed
            i = algorithm_globals.random.integers(0, len(y))

            value = self._compute_weighted_kernel_sum(i, X, training=True)

            if (self._label_map[y[i]] * self.C / step) * value < 1:
                # only way for a component of alpha to become non zero
                self._alphas[i] = self._alphas.get(i, 0) + 1

        self.fit_status_ = PegasosQSVC.FITTED

        logger.debug("fit completed after %s", str(datetime.now() - t_0)[:-7])

        return self


    # pylint: disable=invalid-name
    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Perform classification on samples in X.

        Args:
            X: Features. For a callable kernel (an instance of
               :class:`~qiskit_machine_learning.kernels.BaseKernel`) the shape
               should be ``(m_samples, n_features)``, for a precomputed kernel the shape should be
               ``(m_samples, n_samples)``. Where ``m`` denotes the set to be predicted and ``n`` the
               size of the training set. In that case, the kernel values in X have to be calculated
               with respect to the elements of the set to be predicted and the training set.

        Returns:
            An array of the shape (n_samples), the predicted class labels for samples in X.

        Raises:
            QiskitMachineLearningError:
                - predict is called before the model has been fit.
            ValueError:
                - Pre-computed kernel matrix has the wrong shape and/or dimension.
        """

        t_0 = datetime.now()
        values = self.decision_function(X)
        y = np.array([self._label_pos if val > 0 else self._label_neg for val in values])
        logger.debug("prediction completed after %s", str(datetime.now() - t_0)[:-7])

        return y


    def decision_function(self, X: np.ndarray) -> np.ndarray:
        """
        Evaluate the decision function for the samples in X.

        Args:
            X: Features. For a callable kernel (an instance of
               :class:`~qiskit_machine_learning.kernels.BaseKernel`) the shape
               should be ``(m_samples, n_features)``, for a precomputed kernel the shape should be
               ``(m_samples, n_samples)``. Where ``m`` denotes the set to be predicted and ``n`` the
               size of the training set. In that case, the kernel values in X have to be calculated
               with respect to the elements of the set to be predicted and the training set.

        Returns:
            An array of the shape (n_samples), the decision function of the sample.

        Raises:
            QiskitMachineLearningError:
                - the method is called before the model has been fit.
            ValueError:
                - Pre-computed kernel matrix has the wrong shape and/or dimension.
        """
        if self.fit_status_ == PegasosQSVC.UNFITTED:
            raise QiskitMachineLearningError("The PegasosQSVC has to be fit first")
        if np.ndim(X) != 2:
            raise ValueError("X has to be a 2D array")
        if self._precomputed and self._n_samples != X.shape[1]:
            raise ValueError(
                "For a precomputed kernel, X should be in shape (m_samples, n_samples)"
            )

        values = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            values[i] = self._compute_weighted_kernel_sum(i, X, training=False)

        return values

#changed part is here-it needs an exception handling that if a circuit fails, we dont lose the experiment
    def _compute_weighted_kernel_sum(self, index: int, X: np.ndarray, training: bool) -> float:
        """Helper function to compute the weighted sum over support vectors used for both training
        and prediction with the Pegasos algorithm.

        Args:
            index: fixed index distinguishing some datum
            X: Features
            training: flag indicating whether the loop is used within training or prediction

        Returns:
            Weighted sum of kernel evaluations employed in the Pegasos algorithm
        """
        # non-zero indices corresponding to the support vectors
        support_indices = list(self._alphas.keys())

        # for training
        if training:
            # support vectors
            x_supp = X[support_indices]
        # for prediction
        else:
            x_supp = self._x_train[support_indices]

        try:
            start_time = time.time()
            if not self._precomputed:
                # evaluate kernel function only for the fixed datum and the support vectors
                kernel = self._quantum_kernel.evaluate(X[index], x_supp) + self._kernel_offset
            else:
                kernel = X[index, support_indices]
            elapsed_time = time.time() - start_time
            if elapsed_time > 4:
                # kernel computation took too long, terminate this round
                return 0  # or any other default value
        except Exception as e:
            # log the exception message
            print(f"Exception occurred while computing kernel: {e}")
            # signal that this round failed
            return 0  # or any other default value


        # map the training labels of the support vectors to {-1,1}
        y = np.array(list(map(self._label_map.get, self._y_train[support_indices])))
        # weights for the support vectors
        alphas = np.array(list(self._alphas.values()))
        # this value corresponds to a sum of kernel values weighted by their labels and alphas
        value = np.sum(alphas * y * kernel)

        return value


#enc of change




    @property
    def quantum_kernel(self) -> BaseKernel:
        """Returns quantum kernel"""
        return self._quantum_kernel

    @quantum_kernel.setter
    def quantum_kernel(self, quantum_kernel: BaseKernel):
        """
        Sets quantum kernel. If previously a precomputed kernel was set, it is reset to ``False``.
        """

        self._quantum_kernel = quantum_kernel
        # quantum kernel is set, so we assume the kernel is not precomputed
        self._precomputed = False

        # reset training status
        self._reset_state()

    @property
    def num_steps(self) -> int:
        """Returns number of steps in the Pegasos algorithm."""
        return self._num_steps

    @num_steps.setter
    def num_steps(self, num_steps: int):
        """Sets the number of steps to be used in the Pegasos algorithm."""
        self._num_steps = num_steps

        # reset training status
        self._reset_state()

    @property
    def precomputed(self) -> bool:
        """Returns a boolean flag indicating whether a precomputed kernel is used."""
        return self._precomputed

    @precomputed.setter
    def precomputed(self, precomputed: bool):
        """Sets the pre-computed kernel flag. If ``True`` is passed then the previous kernel is
        cleared. If ``False`` is passed then a new instance of
        :class:`~qiskit_machine_learning.kernels.FidelityQuantumKernel` is created."""
        self._precomputed = precomputed
        if precomputed:
            # remove the kernel, a precomputed will
            self._quantum_kernel = None
        else:
            # re-create a new default quantum kernel
            self._quantum_kernel = FidelityQuantumKernel()

        # reset training status
        self._reset_state()

    def _reset_state(self):
        """Resets internal data structures used in training."""
        self.fit_status_ = PegasosQSVC.UNFITTED
        self._alphas = None
        self._x_train = None
        self._n_samples = None
        self._y_train = None
        self._label_map = None
        self._label_pos = None
        self._label_neg = None


In [ ]:
#PegasosQSVC
import csv
import os
import numpy as np
from sklearn.datasets import make_blobs
datafilename="BotnetDgaDataset_1000.csv"
resultname="result_BotnetDgaDataset_pegasos_1000.txt"
cwd=os.getcwd()
mycsv=cwd+"/"+datafilename
print(mycsv)
def load_data(filepath):
    with open(filepath) as csv_file:
        data_file = csv.reader(csv_file)
        temp = next(data_file)
        n_samples = 1000
        n_features = 7
        data = np.empty((n_samples, n_features))
        target = np.empty((n_samples,), dtype=int)

        for i, ir in enumerate(data_file):
            data[i] = np.asarray(ir[:-1], dtype=np.float64)
            target[i] = np.asarray(ir[-1], dtype=int)

    return data, target
dgafeatures, labels = load_data(mycsv)


/content/BotnetDgaDataset_1000.csv


In [ ]:
#Data preparation for pegasos
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

features = MinMaxScaler(feature_range=(0, np.pi)).fit_transform(dgafeatures)

train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, train_size=700, shuffle=False
)

In [ ]:
#parameter setting for Pegasos
# number of qubits is equal to the number of features
num_qubits = 7

# number of steps performed during the training procedure
tau = 100

# regularization parameter
C = 1000

In [ ]:
#preparing/;
#1-The default fidelity instantiated in FidelityQuantumKernel
#2-A quantum kernel created from ZFeatureMap
from qiskit import BasicAer
from qiskit.circuit.library import ZFeatureMap
from qiskit.utils import algorithm_globals

from qiskit_machine_learning.kernels import FidelityQuantumKernel, QuantumKernel

algorithm_globals.random_seed = 12345

feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=1)

qkernel = FidelityQuantumKernel(feature_map=feature_map)


In [ ]:
print(backend)

qasm_simulator


In [ ]:
#backend=rigetti_qpu_backend
#backend=rigetti_simulator_backend
#backend = Aer.get_backend('qasm_simulator')
#backend = Quantinuum_simulator_backend
from qiskit.circuit.library import RealAmplitudes, ZFeatureMap, ZZFeatureMap, EfficientSU2

num_features =7
feature_map=ZFeatureMap(num_features,reps=1)
qkernel_backend = QuantumKernel(feature_map=feature_map, quantum_instance=backend)

In [ ]:
#running model
#if you use our modified version not the qiskit original version, then comment the below line
from qiskit_machine_learning.algorithms import PegasosQSVC
import time
pegasos_start=time.perf_counter()
pegasos_qsvc = PegasosQSVC(quantum_kernel=qkernel_backend, C=C, num_steps=tau)
# training
pegasos_qsvc.fit(train_features, train_labels)

In [ ]:
# testing
pegasos_score = pegasos_qsvc.score(test_features, test_labels)
pegasos_end=time.perf_counter()

print(f"PegasosQSVC Accuracy: {pegasos_score}")
print(f"PegasosQSVC Time: {pegasos_end-pegasos_start}")

PegasosQSVC Accuracy: 0.8233333333333334
PegasosQSVC Time: 90.52283525499979


In [ ]:
#Warning - this cell is for calculation of average accuracy and time for 10 times running - not possible to do it out of AER due to speed and cost of running
# Setting up the loop to run the PegasosQSVC 10 times and collect accuracies and times

num_runs = 10
pegasos_scores = []
pegasos_times = []

for _ in range(num_runs):
    pegasos_start = time.perf_counter()

    pegasos_qsvc = PegasosQSVC(quantum_kernel=qkernel_backend, C=C, num_steps=tau)
    pegasos_qsvc.fit(train_features, train_labels)
    pegasos_score = pegasos_qsvc.score(test_features, test_labels)

    pegasos_end = time.perf_counter()

    pegasos_scores.append(pegasos_score)
    pegasos_times.append(pegasos_end - pegasos_start)

# Calculating average accuracy and average time
average_accuracy = sum(pegasos_scores) / num_runs
average_time = sum(pegasos_times) / num_runs

average_accuracy, average_time


(0.7393333333333333, 59.373291305700015)